## Testing Google Books Free API

In [1]:
import requests
import pandas as pd

# Prompt user for author name
author_name = input("Enter the name of the author you are looking for: ")

# Replace spaces in author name with "+" for URL encoding
author_name = author_name.replace(" ", "+")

# Make a GET request to the Google Books API to fetch books by author 
response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q=author:{author_name}&langRestrict=en")

# Check the response status code
if response.status_code == 200:

    # Convert the response to JSON
    data = response.json()

    # Extract the list of books from the JSON response
    books = data.get('items', [])

    # Initialize an empty list for book info
    book_info = []

    # Iterating through each book in the list
    for book in books:

        # Access the volume info of the book
        volume_info = book.get('volumeInfo', {})

        # Fetch the required information if available, else assign default values
        title = volume_info.get('title', 'No title available')
        id = book.get('id', 'No ID available')
        subtitle = volume_info.get('subtitle', 'No subtitle available')
        authors = volume_info.get('authors', 'No authors available')
        publishedDate = volume_info.get('publishedDate', 'No publication date available')
        description = volume_info.get('description', 'No description available')
        pageCount = volume_info.get('pageCount', 'No page count available')
        averageRating = volume_info.get('averageRating', 0)
        ratingsCount = volume_info.get('ratingsCount', 0)

        # Appending a dictionary with the gathered information to the list
        book_info.append({'ID': id, 'Title': title, 'Subtitle': subtitle, 'Authors': authors, 'PublishedDate': publishedDate, 'Description': description, 'PageCount': pageCount, 'AverageRating': averageRating, 'RatingsCount': ratingsCount})

    # Create a dataframe from the list of book info
    df = pd.DataFrame(book_info, columns=['ID', 'Title', 'Subtitle', 'Authors', 'PublishedDate', 'Description', 'PageCount', 'AverageRating', 'RatingsCount'])

    # Sorting the dataframe based on AverageRating and RatingsCount
    df = df.sort_values(by=['RatingsCount', 'AverageRating'], ascending=[False, False])

    # Print the top 10 popular books
    print(df[['Title', 'PublishedDate', 'Subtitle', 'RatingsCount', 'Description']].head(10))

elif response.status_code != 200:
    # In case of unsuccessful request, raise an exception
    raise Exception(f"Request failed with status code {response.status_code}. Please try again.")


Enter the name of the author you are looking for: Herman Hesse
                                            Title PublishedDate  \
4                                Klein and Wagner    2017-07-10   
1                                      Siddhartha    2015-03-06   
9                                      Siddhartha    2015-10-06   
0                     The Herman Hesse Collection    2011-07-19   
2           Siddhartha: an Indian Tale(Annotated)    2021-11-29   
3  Siddhartha by Herman Hesse Illustrated Edition    2021-05-25   
5                                       Siddartha    2019-11-05   
6                          The Hesse-Mann Letters       2016-09   
7                                       Siddharta    2021-01-22   
8                        Demian (Spanish Edition)    2017-04-04   

                                            Subtitle  RatingsCount  \
4                              No subtitle available             1   
1                              No subtitle available       

In [2]:
# ...previous code remains the same

top_books = book_info[:min(5, len(book_info))]
for i, book in enumerate(top_books):
    print(f"{i+1}. {book['Title']}")

chosen_book = None

while chosen_book is None:
    try:
        choice = int(input(f"Please enter the number (1-{len(top_books)}) of the book you choose: "))
        chosen_book = top_books[choice - 1]
        print(f"You have chosen: {chosen_book['Title']}")

        # Now print all the details of the chosen book
        print(f"\nDetailed information about your chosen book:\n")
        print(f"ID: {chosen_book['ID']}")
        print(f"Title: {chosen_book['Title']}")
        print(f"Subtitle: {chosen_book['Subtitle']}")
        print(f"Authors: {', '.join(chosen_book['Authors']) if isinstance(chosen_book['Authors'], list) else chosen_book['Authors']}")
        print(f"Published Date: {chosen_book['PublishedDate']}")
        print(f"Description: {chosen_book['Description']}")
        print(f"Page Count: {chosen_book['PageCount']}")

    except (ValueError, IndexError):
        print("Invalid book number entered. Please try again with a valid book number.")

# ...the remaining code stays the same


1. The Herman Hesse Collection
2. Siddhartha
3. Siddhartha: an Indian Tale(Annotated)
4. Siddhartha by Herman Hesse Illustrated Edition
5. Klein and Wagner
Please enter the number (1-5) of the book you choose: 1
You have chosen: The Herman Hesse Collection

Detailed information about your chosen book:

ID: PGIgYAAACAAJ
Title: The Herman Hesse Collection
Subtitle: Demian + Narcissus and Goldmund + Journey to the East
Authors: Herman Hesse
Published Date: 2011-07-19
Description: No description available
Page Count: 0


In [3]:
import openai
from my_secrets import OPENAI_KEY
openai.api_key = OPENAI_KEY

# Use the chosen book's description as the system's message
system_message = {"role": "system", "content": f"You are the most knowledgeable librarian, capable of summarizing books. Title of the book is: '{chosen_book['Title']}' written by '{', '.join(chosen_book['Authors']) if isinstance(chosen_book['Authors'], list) else chosen_book['Authors']}' and its description: '{chosen_book['Description']}'"}

# Define a list of questions to ask about the book
questions = [
    "Give the basic information about the book in the first paragraph", 
    "Give a short and distilled summary",
    "Give a bullet list of major events that happen in this book",
    "Explain why it is significant and finish with a quote from the book",
]

# Build the messages list iteratively
messages = [system_message]
for question in questions:
    user_message = {"role": "user", "content": question}
    messages.append(user_message)

# OpenAI Chat conversation
response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=messages,
  max_tokens=2048,
  temperature=1.0  
)

# Print the generated summary
print(response['choices'][0]['message']['content'].strip())


"The Herman Hesse Collection", as the title suggests, is a compilation of works by German-born Swiss author, Herman Hesse. This collection encapsulates his various works renowned worldwide for his profound exploration of an individual's search for authenticity, self-knowledge and spirituality.

This compendium features various narratives each distinctive yet interconnected. As each story unfolds, they explore different themes revolving around spiritual self-discovery, the human condition, the search for meaning, and understanding of one’s place in the world. Hesse's deep insight into the human soul is astoundingly displayed through his characters and their journeys.

Unfortunately, as this is an anthology, it is taxing to list specific events without specifying which work within the collection we're referring to. The collection might include Hesse's most renowned works like "Siddhartha," "Steppenwolf," "Narcissus and Goldmund, " and "The Glass Bead Game," each of these works carry thei